## **Extracción de Características Textuales (Textual Embeddings)**

En este notebook abordamos la fase de extracción de características para la modalidad de texto (transcripciones de diálogos). Mientras que el audio captura el "cómo" se dice y el vídeo el "gesto" con el que se dice, el texto aporta la semántica y el contexto lingüístico, imprescimdibles en nuestra tarea de detección de **Estrés**.

Al igual que en las modalidades anteriores, no utilizaremos el texto en crudo, sino que emplearemos modelos de lenguaje basados en Transformers para generar embeddings contextuales. A diferencia de técnicas tradicionales como Word2Vec, estos modelos utilizan mecanismos de *self-attention* que permiten que la representación de cada palabra dependa de todas las demás palabras de la frase, capturando matices de estrés y frustración.

Dado que todas las transcripciones de nuestro dataset se encuentran en inglés, se han seleccionado los siguientes modelos preentrenados específicamente en este idioma para realizar un estudio comparativo:

1. **BERT** : Es utilizado como *baseline*. Es el modelo clásico pero efectivo que revolucionó el NLP al procesar el texto de forma bidireccional.

2. **RoBERTa**: Versión mejorada de **BERT**, que optimiza el proceso de entrenamiento y el tamaño de los datos, demostrando mayor eficacia en la captura de matices emocionales en diálogos conversacionales.

3. **DeBERTa-v3**: Representa el estado del arte actual. Su mecanismo de atención permite separar la información del contenido de la posición de la palabra, lo que permite una mayor capacidad para entender estructuras gramaticales complejas bajo tensión.


**NOTA**: Para cada frase, extraeremos los embeddings de la última capa oculta. El resultado será un conjunto de archivos *.npy* con dimensiones $(T, 768)$, donde $T$ es el número de tokens y 768 es la densidad del vector de características (en los 3 modelos).
Al cargar la versión **Base** de los tres modelos, se obtienen esas 768 dimensiones, alineadas con los modelos más robustos en audio (**Wav2Vec2.0**) y visión (**ViT**), que convergen de forma nativa a **768 dimensiones** en sus versiones base cargadas. Esto facilita la creación de espacio latente común, donde las dimensiones de entrada están equilibradas.

### Estrategia de Extracción Multi-Ventana:

Basándonos en los resultados obtenidos del EDA, donde se determinó que el **99% de las transcripciones** en los datasets MELD e IEMOCAP poseen una longitud de **27** y **47** tokens respectivamente, se ha diseñado una estrategia de extracción para cumplir con los requisitos del estudio de ablación metodológico:

1.  **Ventana Corta ($T=32$ tokens):** Respuesta inmediata y palabras clave. El objetivo es evaluar si el modelo es capaz de detectar el estrés basándose únicamente en el inicio de la intervención o en frases cortas.
2.  **Ventana Media ($T=64$ tokens):** Balance estándar, tamaño óptimo extraído del EDA ya que cubre el 100% de las transcripciones en MELD y más del 99% en IEMOCAP, evitando la introducción excesiva de *padding* que sí ocurriría con ventanas mayores (por ejemplo, de 128 o 256) que sería excesivo en nuestro caso.


**NOTA**: Para garantizar la validez de la comparación entre ambos tamaños de ventana, se realizan **inferencias independientes** (`max_length=32`, `max_length=64`) para cada configuración. Esto evita el "ruido contextual" que se introduciría al recortar artificialmente un tensor generado con una ventana mayor, ya que los mecanismos de *self-attention* de los Transformers (BERT, RoBERTa, DeBERTa) generan dependencias bidireccionales entre todos los tokens procesados.

In [1]:
!pip install -q transformers sentencepiece torch tqdm

In [ ]:
# Carga previa de todas las librerías y paquetes necesarios
import os
import torch
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModel
from tqdm.notebook import tqdm
from google.colab import drive
import shutil
import zipfile

# Configuración de GPU/CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Dispositivo: {device}")

Dispositivo: cuda


In [3]:
# Se monta Drive para acceder a los datos y guardar los resultados:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


Traemos los datos al disco del servidor de Colab.

La estructura de directorios final dónde se almacenarán los resultados en Drive es la siguiente:
* **`Proyecto_TFG_Data`**:

    * **`EMBEDDINGS_TEXT_BERT`**: 

        * **`EMBEDDINGS_TEXT_BERT_32`**: Aquí se almacenarán los embeddings de tamaño $(32,768)$ obtenidos de **BERT**.

        * **`EMBEDDINGS_TEXT_BERT_64`**: Aquí se almacenarán los embeddings de tamaño $(64,768)$ obtenidos de **BERT**.

    * **`EMBEDDINGS_TEXT_ROBERTA`**: 

        * **`EMBEDDINGS_TEXT_ROBERTA_32`**: Aquí se almacenarán los embeddings de tamaño $(32,768)$ obtenidos de **RoBERTa**.
        
        * **`EMBEDDINGS_TEXT_ROBERTA_64`**: Aquí se almacenarán los embeddings de tamaño $(64,768)$ obtenidos de **RoBERTa**.

    * **`EMBEDDINGS_TEXT_DEBERTA`**: 

        * **`EMBEDDINGS_TEXT_DEBERTA_32`**: Aquí se almacenarán los embeddings de tamaño $(32,768)$ obtenidos de **DeBERTa**.
        
        * **`EMBEDDINGS_TEXT_DEBERTA_64`**: Aquí se almacenarán los embeddings de tamaño $(64,768)$ obtenidos de **DeBERTa**.

In [31]:
LOCAL_DATA_ROOT = '/content/data' # Directorio local en Colab para almacenar los datos
os.makedirs(LOCAL_DATA_ROOT, exist_ok=True)

# RUTAS LOCALES para guardar los .npy inicialmente (Disco SSD de Coolab):
LOCAL_WORK_DIR = '/content/data_temp_features'
if os.path.exists(LOCAL_WORK_DIR):
    shutil.rmtree(LOCAL_WORK_DIR) # Limpiar si existe de antes
os.makedirs(LOCAL_WORK_DIR, exist_ok=True)

# Rutas de SALIDA EN DRIVE:
# Los embeddings sí se guardarán en Drive, pero los datos de entrada se procesarán desde el almacenamiento local para mayor eficiencia

# BERT:
OUTPUT_DIR_BERT = '/content/drive/MyDrive/Proyecto_TFG_Data/EMBEDDINGS_TEXT/EMBEDDINGS_TEXT_BERT'  # Directorio en Drive para guardar ambos embeddings textuales extraídos de BERT
os.makedirs(OUTPUT_DIR_BERT, exist_ok=True)  # Crear el directorio de salida si no existe

# RoBERTa:
OUTPUT_DIR_ROBERTA = '/content/drive/MyDrive/Proyecto_TFG_Data/EMBEDDINGS_TEXT/EMBEDDINGS_TEXT_ROBERTA'  # Directorio en Drive para guardar ambos embeddings textuales extraídos de RoBERTa
os.makedirs(OUTPUT_DIR_ROBERTA, exist_ok=True)  # Crear el directorio de salida si no existe

# DeBERTa:
OUTPUT_DIR_DEBERTA = '/content/drive/MyDrive/Proyecto_TFG_Data/EMBEDDINGS_TEXT/EMBEDDINGS_TEXT_DEBERTA'  # Directorio en Drive para guardar ambos los embeddings textuales extraídos de DeBERTa
os.makedirs(OUTPUT_DIR_DEBERTA, exist_ok=True)  # Crear el directorio de salida si no existe

# Cargamos el CSV directamente desde Drive:
DATA_ROOT_CSV = '/content/drive/MyDrive/Proyecto_TFG_Data/Multimodal_Stress_Dataset.csv' 

In [32]:
# Carga del dataset global

file_path = DATA_ROOT_CSV
if os.path.exists(file_path):
    df_global = pd.read_csv(file_path)
    display(df_global.head()) 
else:
    print(f"No se encuentra el archivo en {file_path}")

,Utterance_ID,Dialogue_ID,video_path,audio_path,Transcription,duration,split,target_stress,dataset_origin
0,train_dia0_utt0,0,train_splits/dia0_utt0.mp4,MELD_Audio/train_dia0_utt0.wav,also I was the point person on my company's tr...,5.672333,train,0,MELD
1,train_dia0_utt1,0,train_splits/dia0_utt1.mp4,MELD_Audio/train_dia0_utt1.wav,You must've had your hands full.,1.501500,train,0,MELD
2,train_dia0_utt2,0,train_splits/dia0_utt2.mp4,MELD_Audio/train_dia0_utt2.wav,That I did. That I did.,2.919583,train,0,MELD
3,train_dia0_utt3,0,train_splits/dia0_utt3.mp4,MELD_Audio/train_dia0_utt3.wav,So let's talk a little bit about your duties.,2.752750,train,0,MELD
4,train_dia0_utt4,0,train_splits/dia0_utt4.mp4,MELD_Audio/train_dia0_utt4.wav,My duties? All right.,6.464792,train,0,MELD


----
## ***Feed-forward.* Fase de Extracción**.

In [ ]:
# Configuración diccionario con los modelos a utilizar y sus respectivas rutas en Hugging Face:
MODELS_CONFIG = {
    "bert": "bert-base-uncased",
    "roberta": "roberta-base",
    "deberta": "microsoft/deberta-v3-base"
}

# Longitudes de ventana fijadas:
WINDOW_SIZES = [32, 64]

def extract_features(df, model_name, hf_path, max_len, local_output_dir):
    """
    Función para extraer características textuales utilizando un modelo de Hugging Face dado.
    
    Args:
        df (pd.DataFrame): DataFrame con las transcripciones y metadatos.
        model_name (str): Nombre del modelo (e.g., 'bert', 'roberta', 'deberta').
        hf_path (str): Ruta del modelo en Hugging Face.
        max_len (int): Longitud máxima de la ventana de texto (tokens)
        local_output_dir (str): Directorio local donde se guardarán los embeddings extraídos.
    
    Devuelve:
        None (los embeddings se guardan como archivos .npy en el directorio especificado).
    """
    os.makedirs(local_output_dir, exist_ok=True)  # Crear el directorio de salida si no existe  
    #Cargamos el tokenizador y el modelo desde Hugging Face
    tokenizer = AutoTokenizer.from_pretrained(hf_path)
    model = AutoModel.from_pretrained(hf_path).to(device) # Cargamos el modelo en el dispositivo (GPU o CPU)
    model.eval()  # Ponemos el modelo en modo evaluación

    for idx, row in tqdm(df.iterrows(), total=len(df), desc=f"Extrayendo características con {model_name} (max_len={max_len})"):
        file_name = f"{row['Dialogue_ID']}_{row['Utterance_ID']}.npy".replace("/","_")

        file_path = os.path.join(local_output_dir, file_name)

        transcription = row['Transcription']

        # Tokenización y creación de tensores
        inputs = tokenizer(transcription, # Tokenizamos la transcripción
                            return_tensors='pt', # Devolvemos tensores PyTorch
                            truncation=True,  # Truncamos el texto si excede max_len
                            padding='max_length',  # Rellenamos con ceros hasta max_len
                            max_length=max_len) # Establecemos la longitud máxima de la ventana
        inputs = {key: val.to(device) for key, val in inputs.items()}

        # Extracción de características sin calcular gradientes (inferencia):
        with torch.no_grad(): 
            outputs = model(**inputs) # Obtenemos las salidas del modelo (embeddings)
            # Guardamos el embedding de la secuencia completa:
            embedding = outputs.last_hidden_state.squeeze(0).cpu().numpy() 

        # Guardar el embedding como un archivo .npy
        np.save(file_path, embedding)  # Guardamos el embedding en un archivo .npy



In [34]:
# Bucle principal:

for name, hf_path in MODELS_CONFIG.items():
    for max_len in WINDOW_SIZES:
        # Definimos el ID de la tarea (ej: bert_32):
        task_id = f"{name}_{max_len}"

        zip_filename = f"EMBEDDINGS_TEXT_{name.upper()}_{max_len}"

        # Ruta local temporal:
        local_save_path = os.path.join(LOCAL_WORK_DIR, task_id)

        if name == 'bert':
            output_dir = OUTPUT_DIR_BERT 
        elif name == 'roberta':
            output_dir = OUTPUT_DIR_ROBERTA
        elif name == 'deberta':
            output_dir = OUTPUT_DIR_DEBERTA
        
        extract_features(df_global, name, hf_path, max_len, local_save_path)

        # Una vez extraídos los embeddings en el almacenamiento local, los movemos a Drive:
        # Con shutil.make_archive creamos el zip:
        shutil.make_archive(f"/content/{zip_filename}", 'zip', local_save_path)
        shutil.copy(f"/content/{zip_filename}.zip", output_dir)

        # Eliminamos los archivos temporales locales para liberar espacio:
        os.remove(f"/content/{zip_filename}.zip")
        shutil.rmtree(local_save_path)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertModel LOAD REPORT from: bert-base-uncased
Key                                        | Status     |  | 
-------------------------------------------+------------+--+-
cls.predictions.transform.dense.weight     | UNEXPECTED |  | 
cls.seq_relationship.weight                | UNEXPECTED |  | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED |  | 
cls.predictions.transform.dense.bias       | UNEXPECTED |  | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED |  | 
cls.seq_relationship.bias                  | UNEXPECTED |  | 
cls.predictions.bias                       | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Extrayendo características con bert (max_len=32):   0%|          | 0/21219 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

...

Extrayendo características con deberta (max_len=32):   0%|          | 0/21219 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/371M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/198 [00:00<?, ?it/s]

DebertaV2Model LOAD REPORT from: microsoft/deberta-v3-base
Key                                     | Status     |  | 
----------------------------------------+------------+--+-
mask_predictions.dense.weight           | UNEXPECTED |  | 
lm_predictions.lm_head.dense.bias       | UNEXPECTED |  | 
lm_predictions.lm_head.LayerNorm.weight | UNEXPECTED |  | 
mask_predictions.classifier.bias        | UNEXPECTED |  | 
lm_predictions.lm_head.bias             | UNEXPECTED |  | 
lm_predictions.lm_head.LayerNorm.bias   | UNEXPECTED |  | 
mask_predictions.dense.bias             | UNEXPECTED |  | 
mask_predictions.LayerNorm.weight       | UNEXPECTED |  | 
mask_predictions.LayerNorm.bias         | UNEXPECTED |  | 
mask_predictions.classifier.weight      | UNEXPECTED |  | 
lm_predictions.lm_head.dense.weight     | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Extrayendo características con deberta (max_len=64):   0%|          | 0/21219 [00:00<?, ?it/s]

### **VALIDACIÓN FINAL DE INTEGRIDAD DE DATOS**

In [ ]:
# Nos aseguramos primero que Drive esté montado
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# 1. OBTENEMOS EL NÚMERO "GROUND TRUTH" (LO ESPERADO)
DATA_ROOT_CSV = '/content/drive/MyDrive/Proyecto_TFG_Data/Multimodal_Stress_Dataset.csv'

if os.path.exists(DATA_ROOT_CSV):
    df_global = pd.read_csv(DATA_ROOT_CSV)
    EXPECTED_COUNT = len(df_global)
    print(f"TOTAL DE FRASES ESPERADAS (según CSV): {EXPECTED_COUNT}")
else:
    print(f"No encuentro el CSV en {DATA_ROOT_CSV}")
    EXPECTED_COUNT = 0

# 2. DEFINIMOS LA ESTRUCTURA A VALIDAR
# Rutas base donde hemos guardado los zips
BASE_DRIVE = '/content/drive/MyDrive/Proyecto_TFG_Data/EMBEDDINGS_TEXT'
MODELS = ["BERT", "ROBERTA", "DEBERTA"]
WINDOW_SIZES = [32, 64]


print(f"{'ARCHIVO ZIP':<40} | {'ENCONTRADOS':<12} | {'ESTADO':<10}")


all_success = True

# 3. BUCLE DE VERIFICACIÓN
for model in MODELS:
    # Ruta de la carpeta del modelo (ej: .../EMBEDDINGS_TEXT_BERT)
    model_dir = os.path.join(BASE_DRIVE, f"EMBEDDINGS_TEXT_{model}")
    
    for size in WINDOW_SIZES:
        # Nombre del ZIP (ej: EMBEDDINGS_TEXT_BERT_32.zip)
        zip_name = f"EMBEDDINGS_TEXT_{model}_{size}.zip"
        zip_full_path = os.path.join(model_dir, zip_name)
        
        found_count = 0
        status = "MISSING"
        
        if os.path.exists(zip_full_path):
            try:
                with zipfile.ZipFile(zip_full_path, 'r') as zip_ref:
                    # Filtramos solo archivos .npy 
                    file_list = [f for f in zip_ref.namelist() if f.endswith('.npy')]
                    found_count = len(file_list)
                
                if found_count == EXPECTED_COUNT:
                    status = "OK"
                else:
                    status = f" DIFERENCIA ({EXPECTED_COUNT - found_count})"
                    all_success = False
            except zipfile.BadZipFile:
                status = "CORRUPTO"
                all_success = False
        else:
            all_success = False

        print(f"{zip_name:<40} | {found_count:<12} | {status}")


if all_success:
    print("\nTodos los archivos coinciden exactamente con el dataset.")
else:
    print("\nSe encontraron discrepancias.")

TOTAL DE FRASES ESPERADAS (según CSV): 21219
ARCHIVO ZIP                              | ENCONTRADOS  | ESTADO    
EMBEDDINGS_TEXT_BERT_32.zip              | 21219        | OK
EMBEDDINGS_TEXT_BERT_64.zip              | 21219        | OK
EMBEDDINGS_TEXT_ROBERTA_32.zip           | 21219        | OK
EMBEDDINGS_TEXT_ROBERTA_64.zip           | 21219        | OK
EMBEDDINGS_TEXT_DEBERTA_32.zip           | 21219        | OK
EMBEDDINGS_TEXT_DEBERTA_64.zip           | 21219        | OK

Todos los archivos coinciden exactamente con el dataset.
